In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats

In [118]:
path = 'C:/Users/MSI/Desktop/study/Data/기업정보/'
df_final = pd.read_csv(path + '이상치처리전_상관관계변수.csv')

In [119]:
df_del = df_final.drop(columns=['Unnamed: 0','BusinessNum','연차','종업원수','대분류코드','stYear','성장단계','주소'])

In [120]:
columns = df_del.columns
columns

Index(['매출액', '영업이익', '당기순이익(손실)', '당좌자산', '매출원가', '매출채권', '매출총이익', '부채총계',
       '영업외비용', '유동부채', '유동자산', '이익잉여금', '이자비용', '인건비', '자기자본', '자본총계', '자산총계',
       '총자산', '투자자산', '판매비와관리비'],
      dtype='object')

# 이상치 확인

In [121]:
def find_outliers(df,column) :
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    return outliers

In [122]:
for i in columns : 

    print(i,'의 이상치비율', len(find_outliers(df_final,i))/len(df_final[i])*100)

매출액 의 이상치비율 13.712787289834951
영업이익 의 이상치비율 20.188698647745383
당기순이익(손실) 의 이상치비율 20.129569643683478
당좌자산 의 이상치비율 13.347729960409275
매출원가 의 이상치비율 11.746105198210705
매출채권 의 이상치비율 11.553293228443621
매출총이익 의 이상치비율 13.255180214921076
부채총계 의 이상치비율 15.522648979381973
영업외비용 의 이상치비율 15.754023343102471
유동부채 의 이상치비율 15.568923852126074
유동자산 의 이상치비율 13.414571443261863
이익잉여금 의 이상치비율 16.095943236156103
이자비용 의 이상치비율 12.005758650830376
인건비 의 이상치비율 12.527636382333283
자기자본 의 이상치비율 13.856753560594376
자본총계 의 이상치비율 13.859324386857935
자산총계 의 이상치비율 13.699933158517148
총자산 의 이상치비율 13.702503984780709
투자자산 의 이상치비율 10.802611959483778
판매비와관리비 의 이상치비율 13.355442439199958


# 이상치 제거

In [123]:
def remove_outliers(df, columns):
    for column in columns:
        outliers = find_outliers(df, column)
        df = df[~df.index.isin(outliers.index)]
    return df

In [124]:
df_clean = remove_outliers(df_final, columns)

In [125]:
print(df_final.shape)
print(df_clean.shape)

(38898, 28)
(10777, 28)


In [126]:
df_clean.to_csv(path+'이상치제거_상관관계변수.csv')

### 폐업처리 할때필요한 과정

In [40]:
df_max_stYear = df_clean.loc[df_clean.groupby('BusinessNum')['stYear'].idxmax()]

In [41]:
len(df_clean.loc[df_clean['폐업여부'] == '폐업']['BusinessNum'].unique())

1077

In [42]:
df_max_stYear.shape

(3870, 26)

In [43]:
df_max_stYear.loc[df_max_stYear['폐업여부']=='폐업'].shape

(1077, 26)

In [44]:
df_max_stYear.to_csv(path+'폐업여부_이상치모두제거(최근값만).csv')

# 이상치 n개 이상 삭제

In [127]:
# IQR을 사용하여 이상치 찾아내는 함수
def find_outliers_2(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = (df[column] < lower_bound) | (df[column] > upper_bound)
    return outliers

# 여러 컬럼에서 이상치 감지
def detect_outliers(df, columns):
    outliers = pd.DataFrame(index=df.index)
    for column in columns:
        outliers[column] = find_outliers_2(df, column)
    return outliers

# 이상치 감지
outliers = detect_outliers(df_final, columns)

# 각 행에서 이상치 수 계산
outliers_count = outliers.sum(axis=1)

# 이상치가 3개 이상인 행의 인덱스 추출
rows_to_drop = outliers_count[outliers_count >= 3].index

# 이상치가 3개 이상인 행 삭제
df_cleaned = df_final.drop(index=rows_to_drop)

In [128]:
df_cleaned.shape

(29457, 28)

In [129]:
df_cleaned.to_csv(path+'이상치3행이상_상관관계변수.csv')

# 이상치 변환

In [44]:
df_final.columns

Index(['BusinessNum', 'stYear', '매출액', '영업이익', '당기순이익(손실)', '당좌자산', '매출원가',
       '매출채권', '매출총이익', '부채총계', '영업외비용', '유동부채', '유동자산', '이익잉여금', '이자비용',
       '인건비', '자기자본', '자본총계', '자산총계', '총자산', '차입금', '투자자산', '판매비와관리비', '종업원수',
       '대분류코드', '성장단계', '주소'],
      dtype='object')

In [130]:
# 상한값 이상은 상한값으로 하한값값 이하는 하한값으로
def replace_outliers_with_bounds(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 상한값을 넘어가는 이상치를 상한값으로 치환
    df.loc[df[column] > upper_bound, column] = upper_bound
    # 하한값 이하인 이상치를 하한값으로 치환
    df.loc[df[column] < lower_bound, column] = lower_bound
    
    return df

# 각 컬럼에 대해 이상치를 bound값으로 변환
for column in columns:
    df_replace_bound = replace_outliers_with_bounds(df_final, column)

In [131]:
df_replace_bound.to_csv(path+'이상치bound_상관관계변수.csv')

### 매출액별 중앙값 차이 그래프

In [9]:
# 귀무가설 : 정규성을 따른다 
# 대립가설 : 정규성을 따르지 않는다.
stats.normaltest(df1['매출액'].dropna()) # <0.5 -> 귀무가설 기각

NormaltestResult(statistic=92721.11795283046, pvalue=0.0)

In [198]:
# 귀무가설 : 대분류별 중앙값 차이없다
# 대립가설 : 대분류별 중앙값 차이있다
df3 = df1[['대분류코드','매출액']].dropna()
grouped_data = [df3[df3['대분류코드'] == group]['매출액'].dropna() for group in df3['대분류코드'].unique()]
stats.kruskal(*grouped_data)  # <0.5 ->  귀무가설기각 -> 차이가있다.

KruskalResult(statistic=1426.4189429885594, pvalue=3.035081663317058e-292)

### 대분류코드별 중앙값 차이 그래프

In [199]:
median_sales = df1.groupby('대분류코드')['매출액'].median().reset_index()
median_sales

,대분류코드,매출액
0,A,17878616.0
1,B,27190335.0
2,C,2898537.5
3,D,14375888.0
4,E,3070903.0
5,F,2077037.5
6,G,4094056.0
7,H,17625824.0
8,I,7340261.5
9,J,1548064.0


### 성장단계별 중앙값 차이 유무

In [13]:
# 귀무가설 : 성장단계별 중앙값 차이없다
# 대립가설 : 성장단계별 중앙값 차이있다
df3 = df1[['성장단계','매출액']].dropna()
grouped_data = [df3[df3['성장단계'] == group]['매출액'].dropna() for group in df3['성장단계'].unique()]
stats.kruskal(*grouped_data)  # <0.5 ->  귀무가설기각 -> 차이가있다.

KruskalResult(statistic=6028.649568384541, pvalue=0.0)

### 대분류코드별 중앙값 차이 그래프

# 주소별 중앙값 차이 유무

In [14]:
df1['주소1'].unique()

array(['서울', '경기', '인천', '경북', '부산', '충남', '경남', '강원', '충북', '전남', '대전',
       '대구', '울산', '제주', '전북', '광주', nan, '세종'], dtype=object)

In [56]:
def convert_add(row):
    if pd.isnull(row) :
        return np.nan
    elif row in ['서울','인천','경기'] :
        return '수도권'
    else :
        return '비수도권'

In [57]:
df1['주소'] = df1['주소1'].apply(convert_add)

In [58]:
df1['주소'].value_counts()

수도권     5307
비수도권    2589
Name: 주소, dtype: int64

In [20]:
# 귀무가설 : 주소별 중앙값 차이없다
# 대립가설 : 주소별 중앙값 차이있다
df3 = df1[['주소','매출액']].dropna()
grouped_data = [df3[df3['주소'] == group]['매출액'].dropna() for group in df3['주소'].unique()]
stats.kruskal(*grouped_data)  # <0.5 ->  귀무가설기각 -> 차이가있다.

KruskalResult(statistic=985.9365289077671, pvalue=2.0472892172586884e-216)

### 주소별 중앙값 차이 그래프

In [115]:
median_sales = df1.groupby('주소')['매출액'].median().reset_index()
median_sales

,주소,매출액
0,비수도권,1457243.0
1,수도권,2607834.5


In [59]:
df1.columns

Index(['Unnamed: 0', 'BusinessNum', '주소1', '연차', '종업원수', '대분류코드', 'stYear',
       '매출액', '영업이익', '당기순이익(손실)', '당좌자산', '매출원가', '매출채권', '매출총이익', '부채총계',
       '영업외비용', '유동부채', '유동자산', '이익잉여금', '이자비용', '인건비', '자기자본', '자본총계', '자산총계',
       '총자산', '차입금', '투자자산', '판매비와관리비', '성장단계', '주소'],
      dtype='object')

In [60]:
df_final = df1[['BusinessNum', 'stYear', '매출액', '영업이익', '당기순이익(손실)',
           '당좌자산', '매출원가', '매출채권', '매출총이익', '부채총계', '영업외비용', '유동부채', '유동자산',
           '이익잉여금', '이자비용', '인건비', '자기자본', '자본총계', '자산총계', '총자산', '차입금', '투자자산',
           '판매비와관리비', '종업원수', '대분류코드', '성장단계', '주소']]

In [61]:
df_final

,BusinessNum,stYear,매출액,영업이익,당기순이익(손실),당좌자산,매출원가,매출채권,매출총이익,부채총계,...,자본총계,자산총계,총자산,차입금,투자자산,판매비와관리비,종업원수,대분류코드,성장단계,주소
0,1010109319,2019.0,178547.0,12955.0,15587.0,739.0,119689.0,NaN,58858.0,2182.0,...,-289.0,1893.0,1893.0,NaN,NaN,45903.0,1.0,G,25년이하,수도권
1,1010204456,2021.0,934616.0,61533.0,59869.0,914293.0,574345.0,NaN,360271.0,1005562.0,...,63841.0,1069403.0,1069403.0,988331.0,NaN,298738.0,5.0,G,25년이하,수도권
2,1010600385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,C,25년이하,수도권
3,1010607727,2019.0,370418.0,40593.0,31162.0,187476.0,123603.0,0.0,246816.0,184275.0,...,18284.0,202559.0,202559.0,150000.0,0.0,206223.0,1.0,N,NaN,수도권
4,1010709848,2020.0,131542.0,21915.0,21915.0,8246.0,60884.0,0.0,70658.0,47978.0,...,-19662.0,28316.0,28316.0,47978.0,0.0,48743.0,11.0,NaN,25년이하,수도권
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,8988107860,2020.0,467332.0,3290.0,14734.0,211074.0,345157.0,209912.0,122175.0,119596.0,...,228964.0,348559.0,348559.0,105610.0,73469.0,118885.0,9.0,F,10년이하,비수도권
7996,8988702634,2021.0,207717.0,39763.0,37315.0,138778.0,109378.0,22397.0,98340.0,47298.0,...,217027.0,264325.0,264325.0,0.0,NaN,58577.0,5.0,J,10년이하,비수도권
7997,8998607522,2018.0,705737.0,11915.0,10082.0,90309.0,525837.0,43670.0,179900.0,319249.0,...,-98866.0,220383.0,220383.0,297462.0,NaN,167985.0,6.0,M,10년이하,수도권
7998,8998708313,2021.0,6813497.0,133499.0,151376.0,870617.0,6305295.0,608503.0,508202.0,264271.0,...,879583.0,1143854.0,1143854.0,103551.0,NaN,374703.0,10.0,F,10년이하,수도권
